In [1]:
import pandas as pd
from IPython.display import display, HTML

pd.set_option('display.float_format', '{:.2f}'.format)

In [2]:
def display_stats(dataframe, dataframe2=None, order_by="translator_name", heading=None):
    df = dataframe.copy()
    df.rename(columns={"cosine_similarity_vs_target": "similarity_to_old_translation"}, inplace=True)
    df['quality_vs_tb'] = df['cosine_similarity_vs_source'] - df['cosine_similarity_original_translation']
    
    if dataframe2 is not None:
        df2 = dataframe2.copy()
        df2.rename(columns={"cosine_similarity_vs_target": "similarity_to_old_translation"}, inplace=True)
        df2['quality_vs_tb'] = df2['cosine_similarity_vs_source'] - df2['cosine_similarity_original_translation']
    
    def make_quantile(q):
        return lambda x: x.quantile(q)
    
    percentiles = [0.01, 0.1, 0.5, 0.9, 0.99]
    agg_funcs = ['mean', 'min', 'max'] + [make_quantile(q) for q in percentiles]
    agg_names = ['mean', 'min', 'max'] + [f"{int(q*100)}%" for q in percentiles]
    
    quality_stats = df.groupby('translator_name')['quality_vs_tb'].agg(agg_funcs).reset_index().set_axis(
        ['translator_name'] + agg_names, axis=1
    )
    
    similarity_stats = df.groupby('translator_name')['similarity_to_old_translation'].agg(agg_funcs).reset_index().set_axis(
        ['translator_name'] + agg_names, axis=1
    )
    
    if heading:
        display(HTML(f"<h3>{heading}</h3>"))
    
    if dataframe2 is not None:
        quality_stats2 = df2.groupby('translator_name')['quality_vs_tb'].agg(agg_funcs).reset_index().set_axis(
            ['translator_name'] + agg_names, axis=1
        )
        similarity_stats2 = df2.groupby('translator_name')['similarity_to_old_translation'].agg(agg_funcs).reset_index().set_axis(
            ['translator_name'] + agg_names, axis=1
        )
        
        quality_diff = quality_stats.merge(quality_stats2, on='translator_name', suffixes=('', '_2'))
        for col in agg_names:
            quality_diff[col] = quality_diff[col] - quality_diff[col + '_2']
        quality_diff = quality_diff[['translator_name'] + agg_names]
        
        similarity_diff = similarity_stats.merge(similarity_stats2, on='translator_name', suffixes=('', '_2'))
        for col in agg_names:
            similarity_diff[col] = similarity_diff[col] - similarity_diff[col + '_2']
        similarity_diff = similarity_diff[['translator_name'] + agg_names]
        
        display(HTML("<h4>Difference in Quality Versus Translation Bureau</h4>"))
        display(quality_diff.sort_values(order_by, ascending=False))
        
        display(HTML("<h4>Difference in Similarity to Translation Bureau Translation</h4>"))
        display(similarity_diff.sort_values(order_by, ascending=False))
        
    else:
        display(HTML("<h4>Quality of Translations Versus Translation Bureau</h4>"))
        display(quality_stats.sort_values(order_by, ascending=False))
        
        display(HTML("<h4>Similarity to Translation Bureau Translation</h4>"))
        display(similarity_stats.sort_values(order_by, ascending=False))


In [3]:
df_test_no_rules = pd.read_csv("translation_results/translation_comparison_test_no_rules_20250905-1430.csv")
df_test_rules = pd.read_csv("translation_results/translation_comparison_test_rules_20250906-0529.csv")
df_test_rules_v2 = pd.read_csv("translation_results/translation_comparison_test_rules_v2_20250906-2158.csv")
df_train_no_rules = pd.read_csv("translation_results/translation_comparison_train_no_rules_20250905-2228.csv")
df_train_rules = pd.read_csv("translation_results/translation_comparison_train_rules_20250906-1419.csv")
df_train_rules_v2 = pd.read_csv("translation_results/translation_comparison_train_rules_v2_20250907-0649.csv")

df_test_rules_error_data = pd.read_json("translation_results/translation_errors_test_rules_20250906-0529.json").T
df_test_rules_v2_error_data = pd.read_json("translation_results/translation_errors_test_rules_v2_20250906-2158.json").T
df_train_rules_error_data = pd.read_json("translation_results/translation_errors_train_rules_20250906-1419.json").T
df_train_rules_v2_error_data = pd.read_json("translation_results/translation_errors_train_rules_v2_20250907-0649.json").T

test_rules_error_list = df_test_rules_error_data.index.to_list()
test_rules_v2_error_list = df_test_rules_v2_error_data.index.to_list()
train_rules_error_list = df_train_rules_error_data.index.to_list()
train_rules_v2_error_list = df_train_rules_v2_error_data.index.to_list()

for df, error_list in [
    (df_test_no_rules, list()), 
    (df_test_rules, test_rules_error_list), 
    (df_test_rules_v2, test_rules_v2_error_list), 
    (df_train_no_rules, list()), 
    (df_train_rules, train_rules_error_list), 
    (df_train_rules_v2, train_rules_v2_error_list)
]:
    df['error'] = False
    df.iloc[
        [x for sample in error_list for x in range((sample - 1) * 7, sample * 7)], 
        df.columns.get_loc('error')
    ] = True


In [4]:
for x in [test_rules_error_list, test_rules_v2_error_list, train_rules_error_list, train_rules_v2_error_list]:
    print(len(x))

914
937
572
542


 # compare quality under different conditions

In [5]:
display_stats(df_test_no_rules, heading="\nTest Data - no find and replace\n")
display_stats(df_test_rules, heading="\nTest Data - with preferential translations\n")
display_stats(df_test_rules_v2, heading="\nTest Data - with preferential translations - finetuned for find and replace\n")

,translator_name,mean,min,max,1%,10%,50%,90%,99%
6,opus_mt_finetuned,0.04,-0.47,0.64,-0.07,-0.02,0.03,0.13,0.33
5,opus_mt_base,0.05,-0.27,0.63,-0.07,-0.02,0.03,0.15,0.33
4,nllb_3b_base_researchonly,0.05,-0.97,0.65,-0.16,-0.02,0.03,0.15,0.33
3,mbart50_mmt_finetuned,0.05,-0.47,0.63,-0.08,-0.02,0.03,0.14,0.32
2,mbart50_mmt_base,0.05,-0.66,0.67,-0.08,-0.02,0.03,0.15,0.34
1,m2m100_418m_finetuned,0.05,-0.66,0.64,-0.08,-0.02,0.03,0.13,0.32
0,m2m100_418m_base,0.06,-0.76,0.67,-0.11,-0.02,0.04,0.16,0.34


,translator_name,mean,min,max,1%,10%,50%,90%,99%
6,opus_mt_finetuned,0.90,-0.02,1.00,0.54,0.80,0.93,0.98,1.00
5,opus_mt_base,0.90,0.16,1.00,0.59,0.79,0.92,0.98,1.00
4,nllb_3b_base_researchonly,0.89,-0.08,1.00,0.49,0.77,0.91,0.97,1.00
3,mbart50_mmt_finetuned,0.90,-0.02,1.00,0.53,0.80,0.92,0.98,1.00
2,mbart50_mmt_base,0.89,0.19,1.00,0.58,0.79,0.91,0.97,1.00
1,m2m100_418m_finetuned,0.89,-0.02,1.00,0.53,0.79,0.92,0.98,1.00
0,m2m100_418m_base,0.87,0.09,1.00,0.51,0.76,0.89,0.96,0.99


,translator_name,mean,min,max,1%,10%,50%,90%,99%
6,opus_mt_finetuned,0.05,-0.56,0.69,-0.07,-0.02,0.03,0.14,0.34
5,opus_mt_base,0.05,-0.28,0.68,-0.07,-0.02,0.03,0.15,0.36
4,nllb_3b_base_researchonly,0.05,-0.86,0.63,-0.13,-0.02,0.04,0.15,0.35
3,mbart50_mmt_finetuned,0.05,-0.70,0.72,-0.08,-0.02,0.03,0.14,0.34
2,mbart50_mmt_base,0.05,-0.31,0.73,-0.08,-0.02,0.03,0.15,0.36
1,m2m100_418m_finetuned,0.05,-0.78,0.71,-0.08,-0.02,0.03,0.14,0.34
0,m2m100_418m_base,0.06,-0.87,0.71,-0.10,-0.01,0.04,0.16,0.36


,translator_name,mean,min,max,1%,10%,50%,90%,99%
6,opus_mt_finetuned,0.90,0.03,1.00,0.53,0.80,0.93,0.98,1.00
5,opus_mt_base,0.90,0.24,1.00,0.56,0.79,0.92,0.97,1.00
4,nllb_3b_base_researchonly,0.89,-0.06,1.00,0.52,0.78,0.91,0.97,1.00
3,mbart50_mmt_finetuned,0.90,0.02,1.00,0.53,0.79,0.92,0.98,1.00
2,mbart50_mmt_base,0.89,0.29,1.00,0.55,0.78,0.91,0.97,0.99
1,m2m100_418m_finetuned,0.89,0.02,1.00,0.51,0.79,0.92,0.98,1.00
0,m2m100_418m_base,0.87,0.03,1.00,0.51,0.76,0.89,0.96,0.99


,translator_name,mean,min,max,1%,10%,50%,90%,99%
6,opus_mt_finetuned,0.05,-0.56,0.67,-0.07,-0.02,0.03,0.13,0.34
5,opus_mt_base,0.05,-0.35,0.65,-0.07,-0.02,0.03,0.15,0.36
4,nllb_3b_base_researchonly,0.05,-0.93,0.67,-0.13,-0.02,0.04,0.15,0.35
3,mbart50_mmt_finetuned,0.05,-0.56,0.64,-0.07,-0.02,0.03,0.14,0.34
2,mbart50_mmt_base,0.05,-0.55,0.67,-0.08,-0.02,0.04,0.15,0.36
1,m2m100_418m_finetuned,0.05,-0.62,0.68,-0.08,-0.02,0.03,0.14,0.35
0,m2m100_418m_base,0.06,-0.82,0.67,-0.10,-0.02,0.04,0.16,0.37


,translator_name,mean,min,max,1%,10%,50%,90%,99%
6,opus_mt_finetuned,0.90,0.01,1.00,0.53,0.79,0.93,0.98,1.00
5,opus_mt_base,0.90,0.29,1.00,0.57,0.79,0.92,0.97,1.00
4,nllb_3b_base_researchonly,0.89,-0.08,1.00,0.49,0.78,0.91,0.97,1.00
3,mbart50_mmt_finetuned,0.90,0.01,1.00,0.52,0.79,0.92,0.98,1.00
2,mbart50_mmt_base,0.89,0.19,1.00,0.57,0.79,0.91,0.97,0.99
1,m2m100_418m_finetuned,0.89,0.01,1.00,0.52,0.79,0.92,0.97,1.00
0,m2m100_418m_base,0.87,0.03,1.00,0.49,0.76,0.89,0.96,0.99


In [6]:
display_stats(df_train_no_rules, heading="\nTraining Data - no find and replace\n")
display_stats(df_train_rules, heading="\nTraining Data - with preferential translations\n")
display_stats(df_train_rules_v2, heading="\nTraining Data - with preferential translations - finetuned for find and replace\n")

,translator_name,mean,min,max,1%,10%,50%,90%,99%
6,opus_mt_finetuned,0.02,-0.51,0.28,-0.04,-0.01,0.01,0.07,0.15
5,opus_mt_base,0.03,-0.30,0.28,-0.05,-0.01,0.02,0.08,0.16
4,nllb_3b_base_researchonly,0.03,-0.80,0.33,-0.10,-0.01,0.02,0.08,0.16
3,mbart50_mmt_finetuned,0.02,-0.50,0.28,-0.05,-0.01,0.01,0.07,0.15
2,mbart50_mmt_base,0.03,-0.65,0.31,-0.05,-0.01,0.02,0.08,0.16
1,m2m100_418m_finetuned,0.02,-0.69,0.31,-0.05,-0.01,0.02,0.07,0.15
0,m2m100_418m_base,0.03,-0.60,0.34,-0.09,-0.02,0.02,0.08,0.16


,translator_name,mean,min,max,1%,10%,50%,90%,99%
6,opus_mt_finetuned,0.95,0.03,1.00,0.79,0.89,0.96,1.00,1.00
5,opus_mt_base,0.94,0.58,1.00,0.78,0.88,0.95,0.99,1.00
4,nllb_3b_base_researchonly,0.93,-0.03,1.00,0.74,0.87,0.95,0.99,1.00
3,mbart50_mmt_finetuned,0.95,0.03,1.00,0.78,0.89,0.96,0.99,1.00
2,mbart50_mmt_base,0.93,0.21,1.00,0.77,0.87,0.95,0.99,1.00
1,m2m100_418m_finetuned,0.94,0.03,1.00,0.77,0.88,0.96,0.99,1.00
0,m2m100_418m_base,0.92,0.32,1.00,0.74,0.85,0.93,0.98,1.00


,translator_name,mean,min,max,1%,10%,50%,90%,99%
6,opus_mt_finetuned,0.03,-0.26,0.81,-0.04,-0.01,0.02,0.07,0.16
5,opus_mt_base,0.03,-0.25,0.81,-0.05,-0.01,0.02,0.08,0.17
4,nllb_3b_base_researchonly,0.03,-0.79,0.82,-0.10,-0.01,0.02,0.08,0.17
3,mbart50_mmt_finetuned,0.03,-0.38,0.81,-0.04,-0.01,0.02,0.07,0.16
2,mbart50_mmt_base,0.03,-0.74,0.81,-0.05,-0.01,0.02,0.08,0.16
1,m2m100_418m_finetuned,0.03,-0.69,0.79,-0.05,-0.01,0.02,0.07,0.16
0,m2m100_418m_base,0.03,-0.53,0.81,-0.09,-0.02,0.02,0.08,0.17


,translator_name,mean,min,max,1%,10%,50%,90%,99%
6,opus_mt_finetuned,0.95,0.18,1.00,0.79,0.89,0.96,1.00,1.00
5,opus_mt_base,0.94,0.18,1.00,0.77,0.88,0.95,0.99,1.00
4,nllb_3b_base_researchonly,0.93,-0.03,1.00,0.74,0.87,0.95,0.99,1.00
3,mbart50_mmt_finetuned,0.95,0.19,1.00,0.78,0.89,0.96,0.99,1.00
2,mbart50_mmt_base,0.93,0.18,1.00,0.76,0.87,0.94,0.99,1.00
1,m2m100_418m_finetuned,0.94,0.20,1.00,0.77,0.88,0.95,0.99,1.00
0,m2m100_418m_base,0.92,0.20,1.00,0.74,0.85,0.93,0.98,1.00


,translator_name,mean,min,max,1%,10%,50%,90%,99%
6,opus_mt_finetuned,0.02,-0.51,0.34,-0.04,-0.01,0.02,0.07,0.15
5,opus_mt_base,0.03,-0.17,0.34,-0.05,-0.01,0.02,0.08,0.16
4,nllb_3b_base_researchonly,0.03,-0.80,0.34,-0.08,-0.01,0.02,0.08,0.16
3,mbart50_mmt_finetuned,0.03,-0.54,0.34,-0.05,-0.01,0.02,0.07,0.15
2,mbart50_mmt_base,0.03,-0.71,0.33,-0.05,-0.01,0.02,0.08,0.16
1,m2m100_418m_finetuned,0.03,-0.51,0.33,-0.05,-0.01,0.02,0.07,0.15
0,m2m100_418m_base,0.03,-0.74,0.33,-0.08,-0.02,0.02,0.08,0.16


,translator_name,mean,min,max,1%,10%,50%,90%,99%
6,opus_mt_finetuned,0.95,0.22,1.00,0.79,0.89,0.96,1.00,1.00
5,opus_mt_base,0.94,0.61,1.00,0.78,0.88,0.95,0.99,1.00
4,nllb_3b_base_researchonly,0.93,0.15,1.00,0.76,0.87,0.95,0.99,1.00
3,mbart50_mmt_finetuned,0.95,0.22,1.00,0.79,0.89,0.96,0.99,1.00
2,mbart50_mmt_base,0.93,0.25,1.00,0.77,0.87,0.94,0.99,1.00
1,m2m100_418m_finetuned,0.94,0.10,1.00,0.78,0.88,0.95,0.99,1.00
0,m2m100_418m_base,0.92,0.25,1.00,0.74,0.85,0.93,0.98,1.00


# comparison of each model

In [7]:
finetuned_translators = ['opus_mt_finetuned', 'mbart50_mmt_finetuned', 'm2m100_418m_finetuned']

In [8]:
display_stats(
    df_test_rules[df_test_rules.translator_name.isin(finetuned_translators)], 
    df_test_no_rules[df_test_no_rules.translator_name.isin(finetuned_translators)], 
    heading="\nTest Data - find and replace replace minus no find and\n"
)
display_stats(
    df_test_rules_v2[df_test_rules_v2.translator_name.isin(finetuned_translators)], 
    df_test_rules[df_test_rules.translator_name.isin(finetuned_translators)], 
    heading="\nTest Data - finetuned find and replace minus find and replace\n"
)
display_stats(
    df_test_rules_v2[df_test_rules_v2.translator_name.isin(finetuned_translators)], 
    df_test_no_rules[df_test_no_rules.translator_name.isin(finetuned_translators)], 
    heading="\nTest Data - finetuned find and replace minus no find and replace\n"
)

,translator_name,mean,min,max,1%,10%,50%,90%,99%
2,opus_mt_finetuned,0.00,-0.09,0.05,0.00,0.00,0.00,0.00,0.01
1,mbart50_mmt_finetuned,0.00,-0.23,0.09,0.01,0.00,0.00,0.00,0.01
0,m2m100_418m_finetuned,0.00,-0.11,0.07,0.00,0.00,-0.00,0.01,0.02


,translator_name,mean,min,max,1%,10%,50%,90%,99%
2,opus_mt_finetuned,-0.00,0.04,0.00,-0.01,-0.00,-0.00,-0.00,-0.00
1,mbart50_mmt_finetuned,-0.00,0.03,0.00,-0.00,-0.00,0.00,-0.00,-0.00
0,m2m100_418m_finetuned,0.00,0.03,0.00,-0.03,-0.00,0.00,-0.00,-0.00


,translator_name,mean,min,max,1%,10%,50%,90%,99%
2,opus_mt_finetuned,0.00,0.00,-0.02,-0.00,-0.00,0.00,-0.00,0.00
1,mbart50_mmt_finetuned,0.00,0.14,-0.08,0.00,-0.00,0.00,0.00,0.00
0,m2m100_418m_finetuned,0.00,0.16,-0.03,-0.00,0.00,0.00,-0.00,0.00


,translator_name,mean,min,max,1%,10%,50%,90%,99%
2,opus_mt_finetuned,-0.00,-0.02,0.00,-0.00,-0.00,-0.00,-0.00,-0.00
1,mbart50_mmt_finetuned,-0.00,-0.01,0.00,-0.00,-0.00,-0.00,-0.00,0.00
0,m2m100_418m_finetuned,-0.00,-0.01,-0.00,0.01,-0.00,-0.00,-0.00,-0.00


,translator_name,mean,min,max,1%,10%,50%,90%,99%
2,opus_mt_finetuned,0.00,-0.09,0.03,0.00,0.00,0.00,-0.00,0.02
1,mbart50_mmt_finetuned,0.00,-0.09,0.01,0.01,0.00,0.00,0.00,0.02
0,m2m100_418m_finetuned,0.00,0.04,0.04,0.00,0.00,0.00,0.00,0.02


,translator_name,mean,min,max,1%,10%,50%,90%,99%
2,opus_mt_finetuned,-0.00,0.02,0.00,-0.01,-0.01,-0.00,-0.00,-0.00
1,mbart50_mmt_finetuned,-0.00,0.02,0.00,-0.01,-0.00,-0.00,-0.00,-0.00
0,m2m100_418m_finetuned,-0.00,0.02,-0.00,-0.02,-0.00,-0.00,-0.00,-0.00


In [9]:
display_stats(
    df_train_rules[df_train_rules.translator_name.isin(finetuned_translators)],
    df_train_no_rules[df_train_no_rules.translator_name.isin(finetuned_translators)], 
    heading="\nTraining Data - find and replace replace minus no find and\n"
)
display_stats(
    df_train_rules_v2[df_train_rules_v2.translator_name.isin(finetuned_translators)],
    df_train_rules[df_train_rules.translator_name.isin(finetuned_translators)], 
    heading="\nTraining Data - finetuned find and replace minus find and replace\n"
)
display_stats(
    df_train_rules_v2[df_train_rules_v2.translator_name.isin(finetuned_translators)],
    df_train_no_rules[df_train_no_rules.translator_name.isin(finetuned_translators)], 
    heading="\nTraining Data - finetuned find and replace minus no find and replace\n"
)

,translator_name,mean,min,max,1%,10%,50%,90%,99%
2,opus_mt_finetuned,0.00,0.25,0.53,-0.00,-0.00,0.00,0.00,0.01
1,mbart50_mmt_finetuned,0.00,0.12,0.53,0.00,-0.00,0.00,0.00,0.01
0,m2m100_418m_finetuned,0.00,0.00,0.48,-0.00,-0.00,0.00,0.00,0.01


,translator_name,mean,min,max,1%,10%,50%,90%,99%
2,opus_mt_finetuned,-0.00,0.15,0.00,-0.00,-0.00,-0.00,-0.00,0.00
1,mbart50_mmt_finetuned,-0.00,0.16,0.00,-0.00,-0.00,-0.00,-0.00,0.00
0,m2m100_418m_finetuned,-0.00,0.17,0.00,-0.01,-0.00,-0.00,-0.00,0.00


,translator_name,mean,min,max,1%,10%,50%,90%,99%
2,opus_mt_finetuned,-0.00,-0.25,-0.47,-0.00,-0.00,-0.00,-0.00,-0.01
1,mbart50_mmt_finetuned,-0.00,-0.16,-0.47,-0.01,-0.00,0.00,-0.00,-0.01
0,m2m100_418m_finetuned,0.00,0.18,-0.47,0.00,-0.00,0.00,-0.00,-0.00


,translator_name,mean,min,max,1%,10%,50%,90%,99%
2,opus_mt_finetuned,-0.00,0.04,0.00,0.00,0.00,-0.00,-0.00,0.00
1,mbart50_mmt_finetuned,-0.00,0.03,0.00,0.01,0.00,-0.00,-0.00,0.00
0,m2m100_418m_finetuned,0.00,-0.11,0.00,0.01,0.00,-0.00,0.00,0.00


,translator_name,mean,min,max,1%,10%,50%,90%,99%
2,opus_mt_finetuned,0.00,0.00,0.06,-0.00,-0.00,0.00,0.00,-0.00
1,mbart50_mmt_finetuned,0.00,-0.04,0.06,-0.00,-0.00,0.00,0.00,0.00
0,m2m100_418m_finetuned,0.00,0.18,0.02,-0.00,-0.00,0.00,0.00,0.00


,translator_name,mean,min,max,1%,10%,50%,90%,99%
2,opus_mt_finetuned,-0.00,0.19,0.00,0.00,-0.00,-0.00,-0.00,0.00
1,mbart50_mmt_finetuned,-0.00,0.19,0.00,0.01,-0.00,-0.00,-0.00,0.00
0,m2m100_418m_finetuned,-0.00,0.06,0.00,0.01,-0.00,-0.00,-0.00,0.00


# Survey Results

In [ ]:
# TBD

